<a href="https://colab.research.google.com/github/sribarrow/dissertation/blob/master/cnn_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import keras
from keras import backend as K
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import LSTM, Conv2D, MaxPooling2D, Dropout
from keras.layers import Dense, Activation, Flatten
from keras import optimizers, regularizers

Using TensorFlow backend.


Reference: https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=3IEVK-KFxi5Z

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from tensorflow.python.client import device_lib

In [4]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12492246685155605758, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8194939407523460543
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1392948797179046075
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14912199066
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2433809120705814740
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# with open('/gdrive/My Drive/foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# !cat '/gdrive/My Drive/foo.txt'

Reference: https://github.com/tensorflow/tensorflow/issues/17076

Fix for ERROR - report_tensor_allocations_upon_oom

In [0]:
# config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 1} ) 
#  sess = tf.Session(config=config) 
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [8]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [9]:
from tensorflow.core.protobuf import config_pb2
sess.run(c, options=config_pb2.RunOptions(
        report_tensor_allocations_upon_oom=True))

array([[22., 28.],
       [49., 64.]], dtype=float32)

Reference: https://stackoverflow.com/questions/46066850/understanding-the-resourceexhaustederror-oom-when-allocating-tensor-with-shape

Fix for ERROR - ResourceExhaustedError: OOM when allocating tensor with shape

In [0]:
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [0]:
train_path = '/gdrive/My Drive/raag_images/train'
valid_path = '/gdrive/My Drive/raag_images/check'
test_path = '/gdrive/My Drive/raag_images/test'

In [12]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(300,300), classes=['RaagM','RaagS'],  batch_size=32)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path,  target_size=(300,300), classes=['RaagM','RaagS'],  batch_size=4)
test_batches = ImageDataGenerator().flow_from_directory(test_path,  target_size=(300,300), classes=['RaagM','RaagS'],  batch_size=10)

Found 576 images belonging to 2 classes.
Found 26 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [13]:
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [14]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(300,300,3)))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Conv2D(64, (3, 3)))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
model.compile(optimizers.Adam(lr=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()


W0825 21:42:49.916176 140265219471232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0825 21:42:49.920051 140265219471232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0825 21:42:49.926119 140265219471232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0825 21:42:49.979489 140265219471232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0825 21:42:49.987349 140265219471232 deprecation_wrappe

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 300, 300, 32)      896       
_________________________________________________________________
flatten_1 (Flatten)          (None, 2880000)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 5760002   
Total params: 5,760,898
Trainable params: 5,760,898
Non-trainable params: 0
_________________________________________________________________


In [15]:
STEP_SIZE_TRAIN=train_batches.n//train_batches.batch_size
print(STEP_SIZE_TRAIN)
STEP_SIZE_VALID=valid_batches.n//valid_batches.batch_size
print(STEP_SIZE_VALID)


18
6


In [16]:
model.fit_generator(train_batches, steps_per_epoch=STEP_SIZE_TRAIN, epochs=5, validation_data=valid_batches, validation_steps=STEP_SIZE_VALID)

W0825 21:42:50.099367 140265219471232 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0825 21:42:50.144230 140265219471232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/5
18/18 [==============================] - 199s 11s/step - loss: 7.6997 - acc: 0.5191 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/5
18/18 [==============================] - 4s 248ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/5
18/18 [==============================] - 4s 248ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/5
18/18 [==============================] - 5s 252ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 7.3264 - val_acc: 0.5455
Epoch 5/5
18/18 [==============================] - 4s 247ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.7917 - val_acc: 0.4545


In [17]:
model.evaluate_generator(generator=valid_batches, steps=STEP_SIZE_VALID)

[8.059047720649028, 0.5]

**Fine tuning the model**

In [0]:
from keras.layers import Input

In [0]:
from keras.layers.pooling import AveragePooling2D

In [0]:
from keras.applications.vgg16 import VGG16

In [0]:
#vgg16_model = keras.applications.vgg16.VGG16()
#vgg16_model.layers.pop()


In [21]:
vgg16_model = keras.applications.vgg16.VGG16(include_top=False, input_tensor=Input(shape=(300, 300, 3)))

W0825 21:46:27.657322 140265219471232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



58892288/58889256 [==============================] - 1s 0us/step


In [22]:
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0         
__________

Fine tuning Reference: https://www.pyimagesearch.com/2019/06/24/change-input-shape-dimensions-for-fine-tuning-with-keras/

**Print the type of this vgg16 model.**

*Note this is of type model and not sequential*


In [23]:
type(vgg16_model)

keras.engine.training.Model

In [0]:
new_model = vgg16_model.output

In [25]:
new_model = AveragePooling2D(pool_size=(4, 4))(new_model)

W0825 21:46:29.141312 140265219471232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



In [26]:
new_model = Flatten(name="flatten")(new_model)
new_model = Dense(128, activation="relu")(new_model)
new_model = Dropout(0.5)(new_model)
new_model = Dense(2, activation="softmax")(new_model)

W0825 21:46:29.181612 140265219471232 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
from keras.models import Model
model = Model(inputs=vgg16_model.input, outputs=new_model)

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0         
__________

**Transform the above model to sequential**

In [0]:
for l in vgg16_model.layers:
  layer_trainable=False

In [30]:
seq_model = Sequential()
type(seq_model)
print(seq_model)

**Add layers from vgg16 to our model**

In [31]:
for l in model.layers:
  print(l)
  seq_model.add(l)

**Remove output layer and change it to fit previous**

In [32]:
seq_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 75, 75, 256)       295168    
__________

In [0]:
opt = optimizers.Adam(lr=1e-4)

seq_model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [0]:
seq_model.fit_generator(train_batches, steps_per_epoch=STEP_SIZE_TRAIN, epochs=15, validation_data=valid_batches, validation_steps=STEP_SIZE_VALID, verbose=2)

Epoch 1/15
 - 20s - loss: 3.7271e-07 - acc: 1.0000 - val_loss: 8.0728 - val_acc: 0.4545
Epoch 2/15
 - 19s - loss: 1.1171e-06 - acc: 1.0000 - val_loss: 6.9719 - val_acc: 0.5455
Epoch 3/15
 - 19s - loss: 4.0501e-07 - acc: 1.0000 - val_loss: 7.5488 - val_acc: 0.5000
Epoch 4/15
 - 18s - loss: 2.7079e-07 - acc: 1.0000 - val_loss: 8.2499 - val_acc: 0.4545
Epoch 5/15
 - 19s - loss: 5.6520e-07 - acc: 1.0000 - val_loss: 5.4877 - val_acc: 0.6364
Epoch 6/15
 - 19s - loss: 4.2271e-07 - acc: 1.0000 - val_loss: 9.0978 - val_acc: 0.4091
Epoch 7/15
